In [3]:
import sqlite3
import pandas as pd

In [9]:
pd.set_option('display.max_columns', None)
conn = sqlite3.connect('data/basketball.sqlite')
db = conn.cursor()

1. get the expected lum sum value of nba stars
- 2 columns players, value

Add on combine with player attributes to be able to search highest salary under certain limited
- Player_salary ps join Player_Attributes pa on ps.namePlayer= pa.DISPLAY_FIRST_LAST

In [47]:
db.execute(    
'''select ps.namePlayer,sum(ps.value) as salary
from Player_salary ps join Player_Attributes pa on ps.namePlayer= pa.DISPLAY_FIRST_LAST
group by ps.namePlayer
HAVING pa.POSITION ='Forward'
order by salary desc
limit 10''')
rows = db.fetchall()
pd.DataFrame(rows)

,0,1
0,Paul George,211715564.0
1,Giannis Antetokounmpo,203793832.0
2,Brandon Ingram,158253000.0
3,Tobias Harris,147258000.0
4,Khris Middleton,146896552.0
5,Pascal Siakam,136905216.0
6,Kevin Durant,126056700.0
7,LeBron James,124875098.0
8,Gordon Hayward,120000000.0
9,Jimmy Butler,108048600.0


In [57]:
db.execute(    
'''select *
from Player_salary

limit 5''')
rows = db.fetchall()
pd.DataFrame(rows)

,0,1,2,3,4,5,6,7,8,9,10,11
0,2020-21,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
1,2021-22,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
2,2022-23,Atlanta Hawks,Bogdan Bogdanovic,current roster,0,0,1,0,0,0,Guaranteed,18000000.0
3,2023-24,Atlanta Hawks,Bogdan Bogdanovic,current roster,1,0,1,0,0,1,Player Option,18000000.0
4,2020-21,Atlanta Hawks,Brandon Goodwin,current roster,0,0,1,0,0,0,Guaranteed,1701593.0


2. get the team salary of the year
- get the salary of each player in a year from q1
- group by the team of the player
- filter the year

In [68]:
db.execute(    
'''select pa.TEAM_NAME,sum(ps.value) as salary,ps.slugSeason
from Player_salary ps join Player_Attributes pa on ps.namePlayer= pa.DISPLAY_FIRST_LAST
group by  pa.TEAM_NAME
HAVING ps.slugSeason ='2020-21'
order by salary desc
limit 30''')
rows = db.fetchall()
pd.DataFrame(rows)

,0,1,2
0,Bucks,591382544.0,2020-21
1,Nets,577356022.0,2020-21
2,Jazz,561305976.0,2020-21
3,76ers,508213140.0,2020-21
4,Warriors,493057681.0,2020-21
5,Clippers,474742140.0,2020-21
6,Trail Blazers,456295999.0,2020-21
7,Celtics,455126777.0,2020-21
8,Lakers,430323153.0,2020-21
9,Pelicans,419701298.0,2020-21


3. Get the highest winning % of team in history
- first group all the teams from all the matches
- find the winning percentage by counting

Problems:
- I dont need to convert text to date before using the date as comparison
- before case I need to put a <font size=5 color=Green>','</font> so it is like i create a new columns base on condition
- I can simply use avg() instead of sum()/count() to get the average...

In [108]:
db.execute(    
'''
select TEAM_ABBREVIATION_HOME,AVG(winning_count) as winning_percentage
from(
select TEAM_ABBREVIATION_HOME,
    CASE
        WHEN (PTS_HOME>PTS_AWAY) THEN '1'
        ELSE '0'
    END AS winning_count
from game

WHERE GAME_DATE > '2000-01-01') as new_table
group by TEAM_ABBREVIATION_HOME
order by winning_percentage desc
limit 5
''')
rows = db.fetchall()
pd.DataFrame(rows,columns=['team','winning %'])

,team,winning %
0,SAS,0.779954
1,DAL,0.675057
2,UTA,0.666283
3,OKC,0.662791
4,DEN,0.659770


4. which play played the most in the same team

In [20]:
db.execute(    
'''select namePlayer,count(distinct(nameTeam)) as number_team_stay, avg(value/1000000) as salary
from Player_Salary
group by namePlayer
order by number_team_stay desc,salary desc
limit 50''')
rows = db.fetchall()
pd.DataFrame(rows)

,0,1,2
0,Norvel Pelle,3,0.138628
1,Blake Griffin,2,21.070521
2,Andre Drummond,2,14.256113
3,Gorgui Dieng,2,8.793844
4,LaMarcus Aldridge,2,8.652494
5,Nicolas Batum,2,7.187750
6,Alex Len,2,1.761686
7,Patrick Patterson,2,1.517279
8,Ben McLemore,2,1.369062
9,Wayne Ellington,2,1.310282


In [69]:
!git add basketball\ practice.ipynb
!git commit -m 'basketball data seet for sql practice'
!git push origin master

[master 7317937] basketball data seet for sql practice
 1 file changed, 623 insertions(+)
 create mode 100644 basketball practice.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 20 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.23 KiB | 3.23 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To github.com:Aipeni/sql_playgorund.git
   7ed04ea..7317937  master -> master
